# Mask R-CNN 자율주행 주차공간 탐색 인공지능

![](https://github.com/kairess/Mask_RCNN/raw/master/assets/4k_video.gif)

In [ ]:
from IPython.display import HTML
HTML('<video src="https://user-images.githubusercontent.com/5242555/218378705-6b285a69-cbe8-47ef-a4b6-b974f0233ae3.webm" width="800" controls/>')

## TensorFlow 다운그레이드

2.11.0 -> 2.5.0

In [ ]:
import tensorflow as tf
tf.__version__

'2.11.0'

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
!dpkg -i libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.*
!pip install -U -qq tensorflow==2.5.0
exit() # 런타임 다시 시작 필요!

--2023-02-13 04:44:43--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 430460776 (411M) [application/x-deb]
Saving to: ‘libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb’

libcudnn8_8.1.0.77- 100%[===================>] 410.52M   286MB/s    in 1.4s    

2023-02-13 04:44:44 (286 MB/s) - ‘libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb’ saved [430460776/430460776]

dpkg: warning: downgrading libcudnn8 from 8.4.0.27-1+cuda11.6 to 8.1.0.77-1+cuda11.2
(Reading database ... 128048 files and directories currently installed.)
Preparing to unpack libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+cuda11.2) over (8.4.0.27-1+cuda11.6) ...
Setting up libcudnn8 (8.1.0.77-1+c

In [ ]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15546857329933517555, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14476509184
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8543149671718741829
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
!nvidia-smi

Mon Feb 13 04:46:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |    254MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mask R-CNN 소스코드

- Modified for Colab: https://github.com/kairess/Mask_RCNN
- Modified: https://github.com/akTwelve/Mask_RCNN
- Original: https://github.com/matterport/Mask_RCNN

In [ ]:
!git clone https://github.com/kairess/Mask_RCNN

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 1543, done.
remote: Total 1543 (delta 0), reused 0 (delta 0), pack-reused 1543
Receiving objects: 100% (1543/1543), 1.26 GiB | 24.14 MiB/s, done.
Resolving deltas: 100% (596/596), done.
Updating files: 100% (624/624), done.


In [ ]:
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw

ROOT_DIR = 'Mask_RCNN'

sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

## 테스트 데이터 다운로드

**전체 데이터 다운로드 방법**

### 주차 공간 탐색을 위한 차량 관점 복합 데이터 (AI허브)

https://www.aihub.or.kr/aihubdata/data/view.do?dataSetSn=598

<img src="https://www.aihub.or.kr/web-nas/aihub21/files/editor/2022/06/d2d2490af9e3466badefa28bb55af3d7.jpg" width="800px"/>

In [ ]:
gdown.download('https://drive.google.com/uc?id=1aDiMsr--LwXuRUBssqaFyn_xf-CrVE2H')

!unzip -qq parking-space-indoor.zip

Downloading...
From: https://drive.google.com/uc?id=1aDiMsr--LwXuRUBssqaFyn_xf-CrVE2H
To: /content/parking-space-indoor.zip
100%|██████████| 572M/572M [00:13<00:00, 42.8MB/s]


## 사전학습 모델 다운로드

- mask_rcnn_bbox_0100.h5: 38개 객체 검출 (배경 제외)
  - 자동차, 오토바이 종류 등 포함
- mask_rcnn_seg_0100.h5: 2개 객체 검출 (배경 제외)
  - 배경
  - 주차 가능 공간 (Parking space)
  - 주행 가능 공간 (Drivable space)

In [ ]:
!pip install -U --no-cache-dir gdown --pre -qq

import gdown

gdown.download('https://drive.google.com/uc?id=1_zzHykFnYsYSdLaUXU05lfcfKgha2mx-')
gdown.download('https://drive.google.com/uc?id=1fYSofPxj-CdttG-HLewOg1cQct2di7yS')

Downloading...
From: https://drive.google.com/uc?id=1_zzHykFnYsYSdLaUXU05lfcfKgha2mx-
To: /content/mask_rcnn_bbox_0100.h5
100%|██████████| 257M/257M [00:06<00:00, 38.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fYSofPxj-CdttG-HLewOg1cQct2di7yS
To: /content/mask_rcnn_seg_0100.h5
100%|██████████| 256M/256M [00:04<00:00, 53.6MB/s]


'mask_rcnn_seg_0100.h5'

## Load a Model

In [ ]:
from mrcnn.config import Config

class_names = ["BG", "Parking Space", "Drivable Space"]
# class_names = ["BG", "Car", "Van", "Other Vehicle", "Motorbike", "Bicycle", "Electric Scooter", "Adult", "Child", "Stroller", "Shopping Cart", "Gate Arm", 
#                "Parking Block", "Speed Bump", "Traffic Pole", "Traffic Cone", "Traffic Drum", "Traffic Barricade", "Cylindrical Bollard", "U-shaped Bollard", 
#                "Other Road Barriers", "No Parking Stand", "Adjustable Parking Pole", "Waste Tire", "Planter Barrier", "Water Container", "Movable Obstacle", 
#                "Barrier Gate", "Electric Car Charger", "Parking Meter", "Parking Sign", "Traffic Light", "Pedestrian Light", "Street Sign", "Disabled Parking Space", 
#                "Pregnant Parking Space", "Electric Car Parking Space", "Two-wheeled Vehicle Parking Space", "Other Parking Space"]

class InferenceConfig(Config):
    NAME = "bbox"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = len(class_names)
    DETECTION_MIN_CONFIDENCE = 0.9

inference_config = InferenceConfig()

model_path = "/content/mask_rcnn_seg_0100.h5"
# model_path = "/content/mask_rcnn_bbox_0100.h5"

test_model = modellib.MaskRCNN(
    mode="inference", 
    config=inference_config,
    model_dir=model_path)

test_model.load_weights(model_path, by_name=True)

## Run Inference

In [ ]:
import skimage

mask_colors = [
    (0., 0., 0.), # Background
    (0., 1., 0.), # Parking space
    (0., 0., 1.)  # Drivable space
]

real_test_dir = '/content/parking-space-indoor/대형주차장_004/Camera'
image_paths = []

for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in image_paths[:10]:
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)

    results = test_model.detect([img_arr], verbose=1)
    r = results[0]

    colors = tuple(np.take(mask_colors, r['class_ids'], axis=0))

    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                seg_class_names, r['scores'], figsize=(16, 8),
                                colors=colors)

## 동영상 처리

In [ ]:
import cv2
from google.colab import files
from tqdm import tqdm

mask_colors_255 = [
    (0, 0, 0), # Background
    (0, 255, 0), # Parking space
    (0, 0, 255)  # Drivable space
]

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('output.mp4', fourcc, 10, (1920, 1080))

real_test_dir = '/content/parking-space-indoor/대형주차장_004/Camera'
image_paths = []

for filename in sorted(os.listdir(real_test_dir)):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in tqdm(image_paths):
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)

    results = test_model.detect([img_arr])

    rois = results[0]['rois']
    class_ids = results[0]['class_ids']
    scores = results[0]['scores']
    masks = results[0]['masks']

    result_img = img.copy()

    for i, class_id in enumerate(class_ids):
        mask = masks[:, :, i].astype(np.float32)
        mask = (mask * 255).astype(np.uint8)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(result_img, contours, 0, mask_colors_255[class_id], 2)

        x, y, w, h = cv2.boundingRect(contours[0])
        # cv2.rectangle(result_img, (x, y), (x + w, y + h), (255, 255, 255), 2)

    out.write(result_img)

out.release()

files.download('output.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>